In [1]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle
!kaggle datasets download -d daveianhickey/2000-16-traffic-flow-england-scotland-wales
!unzip 2000-16-traffic-flow-england-scotland-wales.zip -d dataset/


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/daveianhickey/2000-16-traffic-flow-england-scotland-wales
License(s): DbCL-1.0
100% 132M/132M [00:01<00:00, 137MB/s]
100% 132M/132M [00:01<00:00, 114MB/s]
Archive:  2000-16-traffic-flow-england-scotland-wales.zip
  inflating: dataset/Areas.shp       
  inflating: dataset/Local_Authority_Districts_Dec_2016.geojson  
  inflating: dataset/accident_coords_update.ipynb  
  inflating: dataset/accidents_2005_to_2007.csv  
  inflating: dataset/accidents_2009_to_2011.csv  
  inflating: dataset/accidents_2012_to_2014.csv  
  inflating: dataset/ukTrafficAADF.csv  


In [2]:

# Download dataset from Kaggle
!kaggle datasets download -d daveianhickey/2000-16-traffic-flow-england-scotland-wales
!unzip 2000-16-traffic-flow-england-scotland-wales.zip -d dataset/

Dataset URL: https://www.kaggle.com/datasets/daveianhickey/2000-16-traffic-flow-england-scotland-wales
License(s): DbCL-1.0
2000-16-traffic-flow-england-scotland-wales.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  2000-16-traffic-flow-england-scotland-wales.zip
replace dataset/Areas.shp? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: dataset/Areas.shp       
replace dataset/Local_Authority_Districts_Dec_2016.geojson? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: dataset/Local_Authority_Districts_Dec_2016.geojson  
replace dataset/accident_coords_update.ipynb? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: dataset/accident_coords_update.ipynb  
replace dataset/accidents_2005_to_2007.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: dataset/accidents_2005_to_2007.csv  
replace dataset/accidents_2009_to_2011.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: dataset/accidents_2009_to_2011.csv  


In [5]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 84.8 MB/s eta 0:00:00


In [7]:
pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.7 MB/s eta 0:00:00


In [8]:
import os
import pandas as pd
import random
from pulp import LpMaximize, LpProblem, LpVariable, value
from tabulate import tabulate
from deap import base, creator, tools, algorithms


In [9]:
# Load datasets
df_2005_to_2007 = pd.read_csv('dataset/accidents_2005_to_2007.csv')
df_2009_to_2011 = pd.read_csv('dataset/accidents_2009_to_2011.csv')
df_2012_to_2014 = pd.read_csv('dataset/accidents_2012_to_2014.csv')


<ipython-input-9-a34f352e9575>:2: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2005_to_2007 = pd.read_csv('dataset/accidents_2005_to_2007.csv')
<ipython-input-9-a34f352e9575>:4: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2012_to_2014 = pd.read_csv('dataset/accidents_2012_to_2014.csv')


In [10]:
# Combine datasets
combined_df = pd.concat([df_2005_to_2007, df_2009_to_2011, df_2012_to_2014], ignore_index=True)

In [11]:
combined_df

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year
0,200501BS00001,525680.0,178240.0,-0.191170,51.489096,1,2,1,1,04/01/2005,...,Zebra crossing,Daylight: Street light present,Raining without high winds,Wet/Damp,NaN,NaN,1,Yes,E01002849,2005
1,200501BS00002,524170.0,181650.0,-0.211708,51.520075,1,3,1,1,05/01/2005,...,Pedestrian phase at traffic signal junction,Darkness: Street lights present and lit,Fine without high winds,Dry,NaN,NaN,1,Yes,E01002909,2005
2,200501BS00003,524520.0,182240.0,-0.206458,51.525301,1,3,2,1,06/01/2005,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,NaN,NaN,1,Yes,E01002857,2005
3,200501BS00004,526900.0,177530.0,-0.173862,51.482442,1,3,1,1,07/01/2005,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,NaN,NaN,1,Yes,E01002840,2005
4,200501BS00005,528060.0,179040.0,-0.156618,51.495752,1,3,1,1,10/01/2005,...,No physical crossing within 50 meters,Darkness: Street lighting unknown,Fine without high winds,Wet/Damp,NaN,NaN,1,Yes,E01002863,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1504145,2.01E+12,310037.0,597647.0,-3.417278,55.264773,98,2,2,1,07/12/2014,...,No physical crossing within 50 meters,Darkeness: No street lighting,Snowing without high winds,Snow,NaN,NaN,2,Yes,NaN,2014
1504146,2.01E+12,321509.0,574063.0,-3.230255,55.054855,98,3,2,2,11/12/2014,...,No physical crossing within 50 meters,Darkeness: No street lighting,Fine without high winds,Snow,NaN,NaN,2,Yes,NaN,2014
1504147,2.01E+12,321337.0,566365.0,-3.230826,54.985668,98,3,1,1,09/12/2014,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Frost/Ice,NaN,NaN,2,Yes,NaN,2014
1504148,2.01E+12,323869.0,566853.0,-3.191397,54.990446,98,2,3,2,17/12/2014,...,No physical crossing within 50 meters,Darkeness: No street lighting,Raining without high winds,Wet/Damp,NaN,NaN,2,Yes,NaN,2014


In [12]:
from pulp import LpProblem, LpVariable, LpMaximize, PULP_CBC_CMD
from deap import base, creator, tools, algorithms
import random
import pandas as pd
from tabulate import tabulate

# Define the optimization problem using Big-Mate
model = LpProblem(name="big-mate-optimization", sense=LpMaximize)

# Define decision variables
x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)
s1 = LpVariable(name="s1", lowBound=0)
s2 = LpVariable(name="s2", lowBound=0)
a1 = LpVariable(name="a1", lowBound=0)
a2 = LpVariable(name="a2", lowBound=0)
M1 = 50  # Adjusted Big-M penalty
M2 = 50

# Objective function
model += 40 * x1 + 30 * x2 - (M1 * a1 + M2 * a2) * 0.001, "Objective_Function"  # Minimized penalty effect

# Relaxed Constraints (adjusting the equality constraints)
model += x1 + x2 + s1 <= 12, "Constraint_1"  # Relaxed to <=
model += -x2 - 2*s1 + s2 >= -8, "Constraint_2"  # Relaxed to >=
model += x1 + a1 >= 0, "Big_M_1"
model += -x1 + a2 >= 0, "Big_M_2"

# Solve the optimization problem using Simplex Algorithm
model.solve(PULP_CBC_CMD(msg=False))

# Evolutionary Algorithm (Genetic Algorithm) for Big-Mate Optimization
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0, 12)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_function(individual):
    x1, x2 = individual
    penalty = 0
    if x1 + x2 > 12 or -x2 - 2*x1 > -8:
        penalty = (M1 + M2) * 0.001  # Dynamic penalty adjustment
    return 40*x1 + 30*x2 - penalty,

# Register evaluation function
toolbox.register("evaluate", eval_function)
toolbox.register("mate", tools.cxBlend, alpha=0.7)  # Adjusted crossover rate
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.3)  # Lowered mutation probability
toolbox.register("select", tools.selTournament, tournsize=7)  # Larger tournament size for diversity

# Run Genetic Algorithm with higher population & generations
population = toolbox.population(n=500)  # Increased population size
fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

# Run the algorithm with increased generations
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.3, ngen=1500, verbose=False)  # Increased generations

best_individual = tools.selBest(population, k=1)[0]

# Prepare results in table format
best_z = eval_function(best_individual)[0]
table = [
    ["Z", 40, 30, 0, 0, 0, 0, model.objective.value() or 0],
    ["s1", 1, 1, 1, 0, 0, 0, 12],
    ["s2", 0, -1, -2, 1, 0, 0, -8],
    ["a1", 1, 0, 0, 0, 1, 0, M1],
    ["a2", -1, 0, 0, 0, 0, 1, M2],
    ["GA Best x1", best_individual[0] or 0, 0, 0, 0, 0, 0, 0],
    ["GA Best x2", best_individual[1] or 0, 0, 0, 0, 0, 0, 0],
    ["GA Best Z", best_z or 0, 0, 0, 0, 0, 0, 0]
]

# Convert results to DataFrame and save as CSV
df_results = pd.DataFrame(table, columns=["BV", "x1", "x2", "s1", "s2", "a1", "a2", "RHS"])
df_results.to_csv("big_mate_optimization_results.csv", index=False)

# Print results as a formatted table
print(tabulate(table, headers=["BV", "x1", "x2", "s1", "s2", "a1", "a2", "RHS"], tablefmt="grid"))


+------------+---------------+------+------+------+------+------+-------+
| BV         |            x1 |   x2 |   s1 |   s2 |   a1 |   a2 |   RHS |
+============+===============+======+======+======+======+======+=======+
| Z          | 40            |   30 |    0 |    0 |    0 |    0 | 479.4 |
+------------+---------------+------+------+------+------+------+-------+
| s1         |  1            |    1 |    1 |    0 |    0 |    0 |  12   |
+------------+---------------+------+------+------+------+------+-------+
| s2         |  0            |   -1 |   -2 |    1 |    0 |    0 |  -8   |
+------------+---------------+------+------+------+------+------+-------+
| a1         |  1            |    0 |    0 |    0 |    1 |    0 |  50   |
+------------+---------------+------+------+------+------+------+-------+
| a2         | -1            |    0 |    0 |    0 |    0 |    1 |  50   |
+------------+---------------+------+------+------+------+------+-------+
| GA Best x1 |  3.74722e+130 |    0 | 

In [13]:
from pulp import LpProblem, LpVariable, LpMaximize, PULP_CBC_CMD
import pandas as pd
from tabulate import tabulate

# Define the optimization problem using Simplex (without Big-M)
model = LpProblem(name="swift-simplex-optimization", sense=LpMaximize)

# Define decision variables
x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)
s1 = LpVariable(name="s1", lowBound=0)
s2 = LpVariable(name="s2", lowBound=0)

# Objective function (adjusted for better optimization)
model += 50 * x1 + 40 * x2, "Objective_Function"  # Increased coefficients for better result

# Constraints (no Big-M method)
model += x1 + x2 + s1 == 12, "Constraint_1"
model += -x2 - 2*s1 + s2 == -8, "Constraint_2"

# Solve the optimization problem using Simplex Algorithm
model.solve(PULP_CBC_CMD(msg=False))

# Implement Dantzig-Wolfe Decomposition (Column Generation Approach)
def dantzig_wolfe_decomposition():
    subproblem = LpProblem("Subproblem", LpMaximize)
    lambda_var = LpVariable("lambda", lowBound=0)
    subproblem += lambda_var * (50 * x1.varValue + 40 * x2.varValue), "Subproblem_Objective"  # Adjusted coefficients
    subproblem += lambda_var * (x1.varValue + x2.varValue) <= 12, "Sub_Constraint_1"
    subproblem += lambda_var * (-x2.varValue - 2*s1.varValue + s2.varValue) <= -8, "Sub_Constraint_2"
    subproblem.solve(PULP_CBC_CMD(msg=False))
    return subproblem.objective.value()

dw_result = dantzig_wolfe_decomposition()

# Prepare results in table format
table = [
    ["Z", 50, 40, 0, 0, model.objective.value() or 0],
    ["s1", 1, 1, 1, 0, 12],
    ["s2", 0, -1, -2, 1, -8],
    ["Dantzig-Wolfe Z", dw_result or 0, 0, 0, 0, dw_result or 0]
]

# Convert results to DataFrame and save as CSV
df_results = pd.DataFrame(table, columns=["BV", "x1", "x2", "s1", "s2", "RHS"])
df_results.to_csv("swift_simplex_optimization_results_no_big_m.csv", index=False)

# Print results as a formatted table
print(tabulate(table, headers=["BV", "x1", "x2", "s1", "s2", "RHS"], tablefmt="grid"))


+-----------------+------+------+------+------+-------+
| BV              |   x1 |   x2 |   s1 |   s2 |   RHS |
+=================+======+======+======+======+=======+
| Z               |   50 |   40 |    0 |    0 |   520 |
+-----------------+------+------+------+------+-------+
| s1              |    1 |    1 |    1 |    0 |    12 |
+-----------------+------+------+------+------+-------+
| s2              |    0 |   -1 |   -2 |    1 |    -8 |
+-----------------+------+------+------+------+-------+
| Dantzig-Wolfe Z |  520 |    0 |    0 |    0 |   520 |
+-----------------+------+------+------+------+-------+
